##  MNIST 필기체 숫자 인식 (CNN)

#### 전처리 (For.GPU)

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try: # Currently, memory growth needs to be the same across GPUs 
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e: # Memory growth must be set before GPUs have been initialized print(e)
        print(e)

1 Physical GPUs, 1 Logical GPUs


#### 전처리

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers,models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images.reshape((60000,28,28,1))
test_images=test_images.reshape((10000,28,28,1))

# 픽셀 값을 0~1 사이로 정규화한다.
train_images, test_images = train_images / 255.0, test_images / 255.0

#### 컨볼루션 신경망 생성

In [3]:
model=models.Sequential()
# 컨볼루션 + 풀링 (특징 추출기-영상에서 edge따위의 특징을 찾아냄)
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1))) # 32개의 커널로 32개의 특징을 찾아냄, 3x3크기의 커널, relu, 입력: 28x28x1
model.add(layers.MaxPooling2D((2,2))) # 풀링 윈도우의 크기는 2x2, 보폭은 1x1, padding은 valid
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu')) # 출력은 (4,4,64)의 텐서 

# 완전 연결 신경망 (특징 분류기-특징이 담긴 1차원 텐서들을 보고 분류 작업을 함 <분류에 가장 적합한 함수 찾기>)
model.add(layers.Flatten()) # Dense layer는 1차원 텐서를 입력 받음 
model.add(layers.Dense(64, activation='relu')) 
model.add(layers.Dense(10,activation='softmax')) # 0~9 분류 

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

#### 컴파일과 훈련

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images,train_labels,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3408 - accuracy: 0.8905
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0489 - accuracy: 0.9840
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0324 - accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0249 - accuracy: 0.9921
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0194 - accuracy: 0.9935


#### 예측

In [14]:
print(model.predict(test_images)[0],test_labels[0])

[2.8383488e-11 7.9138784e-09 5.6943064e-09 1.1837473e-09 6.1226775e-09
 1.0715828e-09 3.7764537e-14 1.0000000e+00 3.9392317e-10 6.6398442e-09] 7
